In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import json
import pandas as pd
import ast
from tqdm import tqdm
from src.models.llm import call
from src.models.utils import *
from src.models.config import get_model_config
from src.translator.translator import Translator

# from prompts import *
trans = Translator()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# model = 'llama3_8b'
# def llm_config_func(llm):
#     llm.temperature = 0
#     llm.max_tokens = 4096
#     return llm
# config = get_model_config(model)

### llama2 7b
# model = 'llama2_7b'
# def llm_config_func(llm):
#     llm.temperature = 0
#     llm.max_tokens = 2048
#     return llm
# config = get_model_config(model)

### gpt3.5
model = 'gpt-3.5-turbo'
def llm_config_func(llm):
    llm.temperature = 0.8
    llm.max_tokens = 4096
    return llm
config = get_model_config(model)

In [10]:
import os

mc_file_paths = []
for filename in os.listdir("src/data_model_cards/claude3"):
    mc_file_paths.append("src/data_model_cards/claude3/"+filename)

f = open(mc_file_paths[1], "r")
mc_text = f.read()

In [13]:
# extracting scientific jargon from English model cards

prompt = [
    """You are a language expert with a focus on machine translation. Your task is to analyze the following model card and identify scientific jargon or terminology within the field of machine learning that might be challenging for machine translation software to accurately translate to Chinese, Russian, Japanese, French, and Arabic.
        Respond in the following format: [Term 1, Term 2, Term 3, ...]""",
        
    """The following text is:
       "{text}"
    """.format(text=mc_text)
]

res = call(
    prompt,
    llm_config_func,
    has_system_prompt=True,
    model_version=model,
    verbose=True,
    **config
)

term_lst = list(map(lambda s: s.strip(), res[1:-1].split(","))) # could be better 

print(res, term_lst)


1
org-q4KwOHTjWKGwZDHrQlsOPcp3
You are a language expert with a focus on machine translation. Your task is to analyze the following model card and identify scientific jargon or terminology within the field of machine learning that might be challenging for machine translation software to accurately translate to Chinese, Russian, Japanese, French, and Arabic.
        Respond in the following format: [Term 1, Term 2, Term 3, ...]
The following text is:
       "# Model Card for albert-large-v2

ALBERT-large-v2 is a version of the ALBERT model, which is a modification of BERT that uses parameter-reduction techniques to lower memory consumption and increase training speed while still achieving state-of-the-art performance on benchmarks like GLUE, RACE, and SQuAD. It has fewer parameters than BERT-large but establishes new state-of-the-art results through design decisions like using a self-supervised loss that focuses on modeling inter-sentence coherence.

## Model Details

### Model Descript

In [12]:
# if terms not in scientific jargon dictionary, add to dictionary with translations

scientific_jargon_dict = trans.read_json_file('src/scientific_jargon_dict.json')
    
tmp_dict = {"Chinese":{},"Russian":{}, "Japanese":{}, "French":{}, "Arabic":{}}
for term in term_lst:
    if term not in scientific_jargon_dict["Chinese"]:
        tmp_dict["Chinese"][term] = ""
        tmp_dict["Russian"][term] = ""
        tmp_dict["Japanese"][term] = ""
        tmp_dict["French"][term] = ""
        tmp_dict["Arabic"][term] = ""

print("Translating using Seamless M4T ...")

trans.translate_term_m4t(tmp_dict, "English", "Chinese")
print("Chinese translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Russian")
print("Russian translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Japanese")
print("Japanese translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "French")
print("French translation complete")
    
trans.translate_term_m4t(tmp_dict, "English", "Arabic")
print("Arabic translation complete")

    
scientific_jargon_dict.update(tmp_dict)
trans.write_file_to_json(scientific_jargon_dict, 'src/scientific_jargon_dict.json')
print("Scientific Jargon Dictionary Updated")

Translating using Seamless M4T ...


Chinese translation complete
Russian translation complete
Japanese translation complete
French translation complete
Arabic translation complete
Scientific Jargon Dictionary Updated
